In [1]:
# imports (esc + m to make cell into markdown)
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from apyori import apriori #have to pip install apyori
import math
from mlxtend.frequent_patterns import fpmax #have to pip install mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
import re

#TODO
#check support/purity measure: what is this?

Honor Code:  THIS CODE IS MY OWN WORK, IT WAS WRITTEN WITHOUT CONSULTING A  TUTOR OR CODE WRITTEN BY OTHER STUDENTS - David Wang

Step 2: Pre Processing

1. Generate a dictionary

In [5]:
# generate a vocabulary (vocab.txt) from paper.txt

# split the data with python

paper_list = []

with open("paper.txt", 'r') as data_file:
    for line in data_file:
        data = line.split()
        paper_list.append(data)

# place all words into a dictionary called vocab.txt

# build list first with all unique words
unique_list = []

for word_line in range(0, len(paper_list)):
    for word in range(1, len(paper_list[word_line])): #start at 1 because 0 would be PaperID
        # check if the word is unique or not
        if paper_list[word_line][word] not in unique_list:
            unique_list.append(paper_list[word_line][word])
            
# write the list into a text file
with open("vocab.txt", 'w') as data_file:
    for word in unique_list:
        data_file.write(word + "\n")
        

2. Tokenize plain text by dictionary

In [23]:
# transofrm each title (each line in paper.txt) into format of...
# [M] (space) [t1]:[c1] (space) [t2]:[c2] (space) ...
# [M] is number of unique terms in title
# [count] is how many times the term appeared in title
# name output title.txt

vocab_list = []
dict_key = []

with open("vocab.txt", 'r') as data_file:
    for line in data_file:
        data = line.split()
        vocab_list.append(data)
        
for i in range(0, len(vocab_list)):
    dict_key.append(i)

tokenized = [] # to store the final list with tokenized counts



for word_line in range(0, len(paper_list)):

    words_in_line = []
    words = paper_list[word_line]
    for i in range(1, len(words)): # get rid of PaperID
        words_in_line.append(words[i])
        
    # find the unique number of words in the line (M)
    unique_list2 = []
    
    # build the unique list of words
    for word in range(0, len(words_in_line)):
        if words_in_line[word] not in unique_list2:
            unique_list2.append(words_in_line[word])
        
                                              
    M = len(unique_list2)
    
    for i in range(0, len(words_in_line)):
        if M < len(words_in_line): #fix this bug where the index (len(unique_list2) is wrong when there are duplicates
            M = M + 1
    
    #use word_in_line.count() to make a counts list, create list of indeces and create list of :
    counts = []
    indeces = []
    colon = []
    
    #get index from vocab_list
    for word in range(0, len(words_in_line)):
        for vocab in range(0, len(vocab_list)):
            vocab_word = " ".join(vocab_list[vocab])
            if (words_in_line[word] == vocab_word):
                indeces.append(vocab) #vocab is the index of where the word is in the the vocabulary list
        
      
        


    #get counts
    for count in range(0, len(words_in_line)):
        counts.append(words_in_line.count(words_in_line[count]))
        colon.append(':')
        
    
    # equalize lengths
    for i in range(0, len(words_in_line)):
        if (len(counts) != len(indeces)):
            indeces.append(0)
   
    
        
    #append everything created into sub_tokenized list by using a zip
    sub_tokenized = []
    for i in range(0, len(words_in_line)):
        sub_tokenized.append(str(indeces[i]) + colon[i] + str(counts[i]))
    
    
    # get rid of commas and brackets
    sub_tokenized = " ".join(map(str, sub_tokenized))
        
    #take method and apply to all lines in paper_list and add to tokenized
    tokenized.append(str(M) + " " + str(sub_tokenized))
    

        
                       
        
        
# apply tokenized to a text file of title.txt

with open("title.txt", 'w') as data_file:
    for token in tokenized:
        data_file.write(token + "\n")




Step 3: Partitioning

1. Assign a topic to each term

2. re-organize terms by topic

In [24]:
#using word-assignments.dat

assign_list = []

with open('word-assignments.dat', 'r') as data_file:
    for line in data_file:
        data = line.split()
        assign_list.append(data)
        
# create the empty text files, made sure that topics range from 0-4 (5 topics)
for i in range(0, 5):
    with open('topic-' + str(i) + '.txt', 'w') as f:
        pass
    

        
for line in range(0, len(assign_list)):
    
    line0_topic = [] # to save individual lines of topics and IDs
    line1_topic = []
    line2_topic = []
    line3_topic = []
    line4_topic = []
    
    for term in range(1, len(assign_list[line])):
        # topic always at index 6 and max is 5 topics (0, 1, 2, 3, 4)
        ID = assign_list[line][term][0:4] # to get its ID 
        
        
        
        if (assign_list[line][term][6] == '0'):
            line0_topic.append(ID)
        elif (assign_list[line][term][6] == '1'):
            line1_topic.append(ID)
        elif (assign_list[line][term][6] == '2'):
            line2_topic.append(ID)
        elif (assign_list[line][term][6] == '3'):
            line3_topic.append(ID)
        elif (assign_list[line][term][6] == '4'):
            line4_topic.append(ID)
                
    
    line0_topic = " ".join(map(str, line0_topic))
    line1_topic = " ".join(map(str, line1_topic))
    line2_topic = " ".join(map(str, line2_topic))
    line3_topic = " ".join(map(str, line3_topic))
    line4_topic = " ".join(map(str, line4_topic))
    
    
    for i in range(0, 5):
        with open('topic-' + str(i) + '.txt', 'a') as f:
            
            #empty list has false condition so can use it to check if line is suppose to be empty
            #effectively get rid of blank lines
            if (i == 0):
                if (line0_topic):
                    f.write(str(line0_topic) + "\n")
            elif (i == 1):
                if (line1_topic):
                    f.write(str(line1_topic) + "\n")
            elif (i == 2):
                if (line2_topic):
                    f.write(str(line2_topic) + "\n")
            elif (i == 3):
                if (line3_topic):
                    f.write(str(line3_topic) + "\n")
            elif (i == 4): 
                if (line4_topic):
                    f.write(str(line4_topic) + "\n")
                
    
        


Step 4: Mining Frequent Patterns for Each Topic


In [67]:
# build method

def frequent_pattern (inputfile, outputfile, min_support):
    topic_list = []

    with open(str(inputfile), 'r') as data_file:
        for line in data_file:
            data = line.split()
            topic_list.append(data)

    # define the model
    # get rid of support values
    # for i in range(0, len(topic_list)):
    #     topic_list[i].pop(0)

    encoder = TransactionEncoder()
    encoded_data = encoder.fit(topic_list).transform(topic_list)
    encoded_df = pd.DataFrame(encoded_data, columns = encoder.columns_)

    # add to a list of items, make min support higher to get more valuable results
    res = fpgrowth(encoded_df, min_support = min_support, use_colnames = True)

    frequent_list =  res.values.tolist()

    #sort the list and clean up appearence
    support_vals = []
    
    # sort the support_vals

    for i in range(0, len(frequent_list)):
        support_vals.append(float(frequent_list[i][0]))
        support_vals.sort(reverse = True)
        
    sort_list = []
        
    for i in range(0, len(support_vals)):
        for j in range(0, len(frequent_list)):
            if (support_vals[i] == frequent_list[j][0]):
                sort_list.append(str(support_vals[i]) + " " + str(' '.join(list(frequent_list[j][1]))))
                
        sort_list[i].replace('[','').replace(']','').replace(',', '')
                
    

    with open("patterns/" + str(outputfile), 'w') as data_file:
        for result in sort_list:
            data_file.write(result + "\n")
            


# decide min_support by percent of all transactions (Question to ponder A)
# we are curious as to how interseting the rule/pattern is. Looking at the entire data and the supports,
# the support value is ~0.01, so it would make sense to set that as the min_support to achieve the most patterns
# while still having enough memory... By testing from high to low support values, a min_support of 0.01 makes the most sense in frequent patterns
# Any min_supports below 0.01 would not be considered frequent patterns.


for file in range(0, 5):
    frequent_pattern("topic-" + str(file) + ".txt", "pattern-" + str(file) + ".txt", min_support = 0.01)
    
    
    

Step 5: Mining Maximal/Closed Patterns

In [64]:
def closed_pattern(inputfile, outputfile):

    topic_list = []

    with open("patterns/" + str(inputfile), 'r') as data_file:
        for line in data_file:
            data = line.split()
            topic_list.append(data)


    # Mining closed patterns from scratch: frequent pattern where all super patterns are less frequent than closed pattern

    # split up support and itemsets
    supports_list = []
    itemset_list = []
    closed_patterns = []

    for i in range(0, len(topic_list)):
        supports_list.append(topic_list[i][0])
        itemset_list.append(topic_list[i][1:])

    for i in range(0, len(itemset_list)):
        focal_itemset = itemset_list[i]
        focal_support = supports_list[i]

        for j in range(0, len(itemset_list)):
            if (j == i):
                continue

            compare_itemset = itemset_list[j]
            compare_support = supports_list[j]

            if (len(focal_itemset) < len(compare_itemset)): #only checking subpatterns
                presence = [k for e in focal_itemset for k in compare_itemset if e in k]
                if (len(presence) > 0): # if focal is a subset of compare
                    if (focal_support <= compare_support):
                        break # if it already does not satisfy the criteria then go to next item
                    else:
                        closed_patterns.append(str(focal_support) + " " + str(focal_itemset))     

            else:
                continue
    
    # append biggest pattern (always satisfies closed patterns)
    for i in range(0, len(itemset_list)):
        freq = itemset_list[np.argmax([len(l) for l in itemset_list])]
        if (itemset_list[i] == freq):
            closed_patterns.append(str(supports_list[i]) + " " + str(itemset_list[i]))
    
    
    # get rid of duplicates  
    closed_patterns_f = []
    for i in closed_patterns:
        if i not in closed_patterns_f:
            closed_patterns_f.append(i)

    #configure to look nicer
    for i in range(0, len(closed_patterns_f)):
        closed_patterns_f[i] = closed_patterns_f[i].replace('[','').replace(']','').replace(',', '')
        
    with open("closed/" + str(outputfile), 'w') as data_file:
        for result in closed_patterns_f:
            data_file.write(result + "\n")
            

        
for file in range(0, 5):
    closed_pattern("pattern-" + str(file) + ".txt", "closed-" + str(file) + ".txt")


In [69]:
# Mining max patterns with FP max
# max patterns: itemset X is max if X is frequent and there exists no frequent super-pattern with X

def max_pattern(inputfile, outputfile, min_support):

    topic_list = []

    with open(str(inputfile), 'r') as data_file:
        for line in data_file:
            data = line.split()
            topic_list.append(data)

    # define the model
    # get rid of support values
    # for i in range(0, len(topic_list)):
    #     topic_list[i].pop(0)

    encoder = TransactionEncoder()
    encoded_data = encoder.fit(topic_list).transform(topic_list)
    encoded_df = pd.DataFrame(encoded_data, columns = encoder.columns_)

    # add to a list of items, make min support higher to get more valuable results
    res = fpmax(encoded_df, min_support = min_support, use_colnames = True)

    max_list =  res.values.tolist()

    #sort the list and clean up appearence
    
    support_vals = []
    
    # sort the support_vals

    for i in range(0, len(max_list)):
        support_vals.append(float(max_list[i][0]))
        support_vals.sort(reverse = True)
        
    sort_list = []
        
    for i in range(0, len(support_vals)):
        for j in range(0, len(max_list)):
            if (support_vals[i] == max_list[j][0]):
                sort_list.append(str(support_vals[i]) + " " + str(' '.join(list(max_list[j][1]))))
                
        sort_list[i].replace('[','').replace(']','').replace(',', '')
            
            

    with open("max/" + str(outputfile), 'w') as data_file:
        for result in sort_list:
            data_file.write(result + "\n")
            
for file in range(0, 5):
    max_pattern("topic-" + str(file) + ".txt", "max-" + str(file) + ".txt", 0.01) #similar minsupp as aprirori





Step 6: Re-rank by purity of patterns

In [320]:
#re-rank patterns from support rankings

    #f(t,p) is support of p in topic-t.txt
    #D(t) is collection of documents where there is at least one word being assigned to topic t
    #|D(t)| is exactly the number of lines in topic-t.txt
    #D(t, t') is union of D(t) and D(t') that is the size of a set
    #Note... |D(t,t') ≠ |D(t)| + |D(t')|

In [126]:
def purity_test(inputfile, outputfile):

    pattern_support = []

    with open("patterns/pattern-0.txt", 'r') as data_file:
        for line in data_file:
            data = line.split()
            pattern_support.append(data)

    # split up support and pattern
    supports = [] #f(t,p)
    patterns = [] #p

    for item in range(0, len(pattern_support)):
        supports.append(pattern_support[item][0])
        patterns.append(pattern_support[item][1:])

    lines = len(supports) #|D(t)|

    #first part of equation

    #calculate log(supports/total lines) for every line
    first = []
    for i in range(0, lines):
        first.append(math.log(float(supports[i])/lines, 2)) #log based 2

    #second part of equation
    # use data from other topic pattern text files to compare the probability of seeing the pattern in other topics


    # make list of all other supports and patterns found in other text files

    other_supports = [] #f(t',p)
    other_patterns = [] #p
    for i in range(0, 5):

        if i == 0: #if it is the current pattern text file being viewed
            continue

        other = []
        with open('patterns/' + 'pattern-' + str(i) + '.txt', 'r') as data_file:
            for line in data_file:
                data = line.split()
                other.append(data)



        for item in range(0, len(other)):
            other_supports.append(other[item][0])
            other_patterns.append(other[item][1:])


    # put the other lists and supports into a equation

    #set union for D(t,t')
    union = list(set().union(supports, other_supports))
    other_lines = len(union)

    # multiple lists for different patterns
    multiple_patterns = {}
    for i in range(0, len(patterns)):

        #find max of numerator
        multiple_patterns['l'+str(i)] = [] #now there are 5 lists for each type of pattern (using dictionary)
        list_num = []

        #build the dictionary of multiple patterns
        for j in range(0, len(other_patterns)):
            if (patterns[i] == other_patterns[j]):
                num = (float(supports[i]) + float(other_supports[j]))/other_lines #numerator equation

                multiple_patterns['l' + str(i)].append(num)

    #find the max of each dictionary and put into new list called second 
    second = []
    for i in range(0, len(multiple_patterns)):
        if not multiple_patterns['l' + str(i)]:
            max_num = 0.001 #equate as closest possible to 0 for empty sets that do not have other supports
        else:
            max_num = max(multiple_patterns['l' + str(i)])

        second.append(math.log(max_num, 2))


    #subtract all first from second to get purity for each pattern
    purities = []

    for i in range(0, len(patterns)):
        purities.append(first[i] - second[i])

    # add purities and support to the text list with the pattern as well
    textoutput = []

    for i in range(0, len(patterns)):
        patterns[i] = (' '.join(patterns[i]))
        textoutput.append(str(purities[i]) + " " + str(supports[i]) + " " + str(patterns[i]))


    # convert to pandas to sort
    df = pd.DataFrame(columns = ("Purity", "Support", "Patterns"))
    df["Purity"] = purities
    df["Support"] = supports
    df["Patterns"] = patterns


    # noticed that the ones that appear more commonly have a higher purity so need to do subtraction from support - purity for sorting purposes
    df["Purity"] = df["Purity"] * -1

    df.sort_values(by = ["Purity", "Support"], ascending = False, inplace = True)
    df.drop("Support", axis = 1, inplace = True) # get rid of support after sorted

    purity_results = df.values.tolist()

    # write to a text file

    with open("purity/" + str(outputfile), 'w') as data_file:
        for purity in purity_results:
            data_file.write(str(purity) + "\n")
            
            
# run method on all files in pattern

for file in range(0, 5):
    purity_test("pattern-" + str(file) + ".txt", "purity-" + str(file) + ".txt")
    


    
    
    
    

In [45]:

def convert_vocab(inputfile, outputfile):
    
    # correlate pattern with words from vocab_list

    vocab_list = []

    with open("vocab.txt", 'r') as data_file:
        for line in data_file:
            data = line.split()
            vocab_list.append(data)

    pattern_list = []

    with open('patterns/' + str(inputfile) + '.txt', 'r') as data_file:
        for line in data_file:
            data = line.split()
            pattern_list.append(data)

    list_converted = []

    for i in range(0, len(pattern_list)):
        if (len(pattern_list[i]) == 2):
            list_converted.append(vocab_list[int(pattern_list[i][1])])
        else:
            # only other case is when it has 2 words in one pattern
            list_converted.append(vocab_list[int(pattern_list[i][1])] + vocab_list[int(pattern_list[i][2])])
    
    # change to text file
    
    
    with open("patterns/" + str(outputfile), 'w') as data_file:
        for vocab in list_converted:
            data_file.write(str(vocab) + "\n")
            
    
# run method on all files in pattern

for file in range(0, 5):
    convert_vocab("pattern-" + str(file), "pattern-" + str(file) + ".txt.phrase")
    
    



        
